In [1]:
import warnings
warnings.filterwarnings("ignore")

import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# Read csv
df_data = pd.read_csv('E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\complete_dataset_labeled_wLastPurchaseDates.csv', sep='|')
#df_data.head()

In [ ]:
#df_data.info()

In [ ]:
# Convert categories to data type category
#df_data["categories"] = df_data["categories"].astype("category")

# Preprocessing

In [3]:
# Drop columns
df_data.drop('Unnamed: 0', axis=1, inplace=True)
df_data.drop('purchaseDates', axis=1, inplace=True)
df_data.drop('date', axis=1, inplace=True)
df_data.drop('order', axis=1, inplace=True)

df_data.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,week,lastPurchaseDate
0,38769,3477,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]",1,2222-03-03
1,42535,30474,193,10,3,229,3,132,"[3459, 3738, 679, 1628, 4072]",1,2222-03-03
2,42535,15833,1318,4,1,455,0,108,"[2973, 2907, 2749, 3357]",1,2222-03-03
3,42535,20131,347,4,0,291,3,44,"[30, 1515, 1760, 2932, 1287, 2615, 3727, 2450,...",1,2222-03-03
4,42535,4325,539,6,0,303,0,45,"[3104, 1772, 2029, 1274, 3915, 888, 1118, 3882...",1,2222-03-03


### Split datetime into separate columns

In [4]:
# Convert 'lastPurchaseDate' to datetime
df_data['lastPurchaseDate'] = pd.to_datetime(df_data['lastPurchaseDate'], format="%Y-%m-%d")

# Split 'date' into seperate columns
df_data['day'] = df_data['lastPurchaseDate'].dt.day
df_data['month'] = df_data['lastPurchaseDate'].dt.month
df_data['year'] = df_data['lastPurchaseDate'].dt.year

df_data.drop('lastPurchaseDate', axis=1, inplace=True)

In [ ]:
"""
# Move new columns to the front
col = df_data.pop("year")
df_data.insert(1, col.name, col)

col = df_data.pop("month")
df_data.insert(2, col.name, col)

col = df_data.pop("day")
df_data.insert(3, col.name, col)
"""

In [ ]:
# Show table
#df_data.head()

### Multi-Hot-Encoding for categories

In [5]:
# If you run out of memory while encoding the whole dataframe at once you can 
# split the dataframe into chunks beforehand 

df_data_chunk1 = df_data.iloc[:50000,:]
#df_chunk2 = df.iloc[500000:,:]

# Replace symbols in 'categories'
df_data_chunk1['categories'] = df_data_chunk1['categories'].str.replace("[", "")
df_data_chunk1['categories'] = df_data_chunk1['categories'].str.replace("]", "")

In [ ]:
df_data_chunk1.head()

In [6]:
# WARNING: This Cell takes a couple of minutes to compute; ~50GB RAM needed to compute without chunks

# Convert categorical variable into dummy/indicator variables
df_multi_hot = df_data_chunk1["categories"].str.get_dummies(',')
df_multi_hot

,1,1000,1001,1002,1003,1004,1006,1007,1008,1009,...,933,945,955,956,968,970,975,981,990,996
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Combine df_data and df_multi_hot
#pd.merge(df_data_chunk1, df_multi_hot, how='inner', left_index=True, right_index=True)
df_combined = df_data_chunk1.join(df_multi_hot, how='inner')

# delete df_multi_hot to free up RAM
#del df_multi_hot

In [ ]:
#df_combined.info()

In [ ]:
df_combined.head()

In [8]:
#df_combined_chunk1 = df_combined.iloc[:500000,:]
col = df_combined.pop("week")
df_combined.insert(3542, col.name, col)

#df_combined_chunk2 = df_combined.iloc[500000:,:]

In [ ]:
df_combined.head()

In [ ]:
# Check if we have any missing values
df_combined[df_combined.isnull().any(axis=1)]

In [ ]:
"""
For Testing purposes

# Read csv
df_cat = pd.read_csv('E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\complete_dataset_cleaned_labeled.csv', sep='|')
df_cat["categories"] = df_cat["categories"].astype("string")

df_cat.drop('Unnamed: 0', axis=1, inplace=True)
df_cat.drop('date', axis=1, inplace=True)
df_cat.drop('userID', axis=1, inplace=True)
df_cat.drop('itemID', axis=1, inplace=True)
df_cat.drop('order', axis=1, inplace=True)
df_cat.drop('brand', axis=1, inplace=True)
df_cat.drop('feature_1', axis=1, inplace=True)
df_cat.drop('feature_2', axis=1, inplace=True)
df_cat.drop('feature_3', axis=1, inplace=True)
df_cat.drop('feature_4', axis=1, inplace=True)
df_cat.drop('feature_5', axis=1, inplace=True)
df_cat.drop('week', axis=1, inplace=True)

df_cat = df_cat.head(3)
df_cat
"""

### Datatypes for XGBoost

XGBoost natively supports continuous data but not categorical data. In order to use categorical data with XGBoost, we have to use One-Hot-Encoding which converts a column of categorical values into muliple columns of binary values.

# Modeling

In [9]:
df_combined.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,day,...,945,955,956,968,970,975,981,990,996,week
0,38769,3477,186,6,0,196,0,45,"74, 4109, 3867, 803, 4053",3,...,0,0,0,0,0,0,0,0,0,1
1,42535,30474,193,10,3,229,3,132,"3459, 3738, 679, 1628, 4072",3,...,0,0,0,0,0,0,0,0,0,1
2,42535,15833,1318,4,1,455,0,108,"2973, 2907, 2749, 3357",3,...,0,0,0,0,0,0,0,0,0,1
3,42535,20131,347,4,0,291,3,44,"30, 1515, 1760, 2932, 1287, 2615, 3727, 2450, ...",3,...,0,0,0,0,0,0,0,0,0,1
4,42535,4325,539,6,0,303,0,45,"3104, 1772, 2029, 1274, 3915, 888, 1118, 3882,...",3,...,0,0,0,0,0,0,0,0,0,1


In [10]:
df_combined.drop('categories', axis=1, inplace=True)
df_combined.drop('year', axis=1, inplace=True)
df_combined.drop('month', axis=1, inplace=True)
df_combined.drop('day', axis=1, inplace=True)


In [17]:
df_combined.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,1,1000,...,945,955,956,968,970,975,981,990,996,week
0,38769,3477,186,6,0,196,0,45,0,0,...,0,0,0,0,0,0,0,0,0,1
1,42535,30474,193,10,3,229,3,132,0,0,...,0,0,0,0,0,0,0,0,0,1
2,42535,15833,1318,4,1,455,0,108,0,0,...,0,0,0,0,0,0,0,0,0,1
3,42535,20131,347,4,0,291,3,44,0,0,...,0,0,0,0,0,0,0,0,0,1
4,42535,4325,539,6,0,303,0,45,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
# Comma is being used to extract a specific column from a 2D array.
# X = data.iloc[:,:-1]
# X = all rows, all columns except the last one 

X = df_combined.iloc[:,0:-1]
X

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,1,1000,...,933,945,955,956,968,970,975,981,990,996
0,38769,3477,186,6,0,196,0,45,0,0,...,0,0,0,0,0,0,0,0,0,0
1,42535,30474,193,10,3,229,3,132,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42535,15833,1318,4,1,455,0,108,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42535,20131,347,4,0,291,3,44,0,0,...,0,0,0,0,0,0,0,0,0,0
4,42535,4325,539,6,0,303,0,45,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,36106,19863,1058,6,1,503,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,2349,30725,539,10,0,345,0,99,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,2349,2382,618,6,0,181,0,17,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,2349,13421,1180,4,3,322,0,144,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
y = df_combined.iloc[:,-1]
y

0        1
1        1
2        1
3        1
4        1
        ..
49995    1
49996    1
49997    1
49998    1
49999    4
Name: week, Length: 50000, dtype: int64

In [13]:
# Split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [18]:
X_train

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,1,1000,...,933,945,955,956,968,970,975,981,990,996
2660,40703,32565,1232,4,0,370,0,89,0,0,...,0,0,0,0,0,0,0,0,0,0
32815,8968,32015,1111,4,0,486,0,144,0,0,...,0,0,0,0,0,0,0,0,0,0
35141,3003,26195,618,4,0,55,3,11,0,0,...,0,0,0,0,0,0,0,0,0,0
19390,41093,8243,194,10,0,502,0,178,0,0,...,0,0,0,0,0,0,0,0,0,0
34846,45288,12251,1338,4,0,486,3,48,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7763,6378,31515,539,6,0,4,3,45,0,0,...,0,0,0,0,0,0,0,0,0,0
15377,45243,19006,430,10,0,400,0,156,0,0,...,0,0,0,0,0,0,0,0,0,0
17730,45231,20131,347,4,0,291,3,44,0,0,...,0,0,0,0,0,0,0,0,0,0
28030,44054,31153,653,10,1,492,0,39,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X_test

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,1,1000,...,933,945,955,956,968,970,975,981,990,996
11872,37532,11188,6,4,0,71,0,53,0,0,...,0,0,0,0,0,0,0,0,0,0
40828,37211,30599,762,10,2,525,3,84,0,0,...,0,0,0,0,0,0,0,0,0,0
36400,7120,16085,1111,10,0,359,3,132,0,0,...,0,0,0,0,0,0,0,0,0,0
5166,45348,10203,974,6,0,444,3,117,0,0,...,0,0,0,0,0,0,0,0,0,0
30273,23042,26804,685,4,0,538,0,44,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5703,20,18613,1111,4,3,444,3,11,0,0,...,0,0,0,0,0,0,0,0,0,0
36992,369,5391,580,10,0,503,0,17,0,0,...,0,0,0,0,0,0,0,0,0,0
14005,43736,6633,147,4,0,491,3,66,0,0,...,0,0,0,0,0,0,0,0,0,0
29455,2119,10646,810,10,0,509,0,17,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data_dmatrix = xgb.DMatrix(data=X, label=y)

In [15]:
model1 = XGBClassifier()
model1.fit(X_train, y_train)

[22:18:15] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=24,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [16]:
model1.score(X_train,y_train), model1.score(X_test,y_test)

(0.724375, 0.7259)

In [46]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier = classifier.fit(X_train,y_train)

In [47]:
#prediction for decision tree
y_pred1 = classifier.predict(X_test)#Accuracy
from sklearn import metrics
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred1))

Accuracy Score: 0.5905


In [48]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred1)
cm

array([[5697,  547,  440,  584],
       [ 642,   65,   31,   80],
       [ 618,   45,   49,   80],
       [ 890,   69,   69,   94]], dtype=int64)

In [49]:
#prediction for XGBoost
y_pred2 = model1.predict(X_test)#Accuracy
from sklearn import metrics
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred2))

Accuracy Score: 0.7259


In [50]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred2)
cm

array([[7253,    2,    4,    9],
       [ 812,    1,    2,    3],
       [ 790,    0,    1,    1],
       [1116,    0,    2,    4]], dtype=int64)

In [21]:
# make predictions for test data
y_pred = model1.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 72.59%


In [ ]:
model = XGBClassifier(learning_rate=0.1)
model.fit(X_train, y_train)

In [ ]:
model.score(X_train,y_train), model.score(X_test,y_test)

In [ ]:
#model = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
#model.fit(X_train, y_train)

In [ ]:
#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
#xg_reg.fit(X_train,y_train)

#preds = xg_reg.predict(X_test)

In [ ]:
# Je größer der RMSE ist, desto schlechter ist die Anpassung des Modells.
#rmse = np.sqrt(mean_squared_error(y_test, preds))
#print("RMSE: %f" % (rmse))

In [ ]:
#!conda list